## Training default

In [ ]:
from ultralytics import YOLO
import torch
import os

"""
BEVDetNet params:
epochs: 50, 60, 80, 120
batch_size: 8, 16, 24, 32, 48
imgsz: 640x640x3
optimizer: Adam
loss weights: 0.95, 0.98, 1
"""

# Set expandable_segments to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

# Build a new model from scratch and check model specific values
print('--Check YOLO intern values for custom purposes--')
model = YOLO('yolov8s-obb.yaml', task='obb', verbose=False) # n/s/m/l/x
#model = YOLO('yolov8s-obb.yaml', task='obb', verbose=False).load('yolov8s-obb.pt') # test pretrained weights

# Resume to training e.g. run_84 last.pt
#model= YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/run_84/weights/last.pt', 'obb', verbose=False)

# Print layers, parameters, gradients, GFLOPS (computation depends on https://github.com/ultralytics/ultralytics/issues/17547#issuecomment-2481925742)
# source: https://github.com/ultralytics/ultralytics/issues/14749
print(model.info(detailed=False, verbose=True))

# https://community.ultralytics.com/t/about-yolo-configuration-file-yaml/300
results = model.train(data='/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', epochs=50,
                      time=None, patience=0, batch=4, imgsz=640, save=True, save_period=25, cache=True,
                      device=[0, 1], workers=8, project='kitti_bev_yolo', name='b_lr_test', exist_ok=False, 
                      pretrained=False, optimizer='Adam', seed=0, deterministic=False, single_cls=False,
                      classes=None, rect=False, multi_scale=True, cos_lr=False, close_mosaic=0, resume=False, amp=False,
                      fraction=1.0, profile=False, freeze=None, lr0=0.01, lrf=0.01,  momentum=0.937,
                      weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, 
                      box=7.0, cls=0.3, dfl=0.9, pose=0.0, kobj=0.0, nbs=64, overlap_mask=False, mask_ratio=0, # dflt: 7.5, 0.4, 1.0
                      dropout=0.0, val=True, plots=True,
                      
                      hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,
                      flipud=0.0, fliplr=0.0, bgr=0.0, mosaic=0.0, mixup=0.0, copy_paste=0.0, copy_paste_mode='mixup', auto_augment=None,
                      erasing=0.0, crop_fraction=0.0)

# Top training results reached on KITTI LiDAR BEV images for Car, (Pedestrian and Cyclist)
# Adam, batch=4, multi_scale=True, mosaic=0.0, reg_max=12, epoch=50, box=7.5, cls=0.4, dfl=1.0 -> 88.8%/87.2%
# second run -> 89.2%/87.9
# AdamW, batch=16, multi_scale=True, reg_max=16, mosaic=0.8, epoch=80, box=7.5, cls=0.4, dfl=1.0 -> 86.5%/85.7%

# Optional: 
# by default you'll get the final validation confusion matrix with 'results.confusion_matrix.matrix'